# Import dependencies

In [1]:
import sys
import pathlib
from IPython.display import Audio
from typing import List

import librosa
import numpy as np
import pandas as pd
from langdetect import detect
import deepcut
from string import punctuation
import re

ROOT_DIR = pathlib.Path.cwd().parent
sys.path.insert(0, str(ROOT_DIR))
from src.normalizers import EnglishTextNormalizer
from src.evaluate_utils import isd
from src.post_processer import remove_longest_repeating_words

2023-11-24 15:08:39.479511: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-24 15:08:39.844591: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-24 15:08:39.844734: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-24 15:08:39.845851: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-24 15:08:40.030017: I tensorflow/core/platform/cpu_feature_g

# Load result and ground truth

In [2]:
target_folder = ROOT_DIR / 'data/result/Thonburian-whisper-speechbrain-vad' / 'th_Oppday_Q2_2023_IP_interfama'
label_dir = target_folder / 'Groundtruth.csv'
th_result_dir = target_folder / 'th_result.csv'
en_result_dir = target_folder / 'en_result.csv'

In [3]:
label_df = pd.read_csv(str(label_dir))
th_result_df = pd.read_csv(str(th_result_dir)) 
en_result_df = pd.read_csv(str(en_result_dir))

In [4]:
merge_result_df = pd.concat(
    [th_result_df, en_result_df],
    axis=0
)
sorted_result_df = merge_result_df.sort_values('start')
sorted_result_df.to_csv(str(target_folder / 'en-th_result.csv'))

In [5]:
prediction = sorted_result_df['text'].to_list()
# clean_ground_truth_df = pd.read_csv("../test_label_by_best.csv")
# references = clean_ground_truth_df['text'].to_list()
references = label_df['text'].to_list()

In [6]:
def en_preprocess(text: str) -> str:
    text = re.sub("[QA]:|(uh)","", text)
    return text

def th_preprocess(text: str) -> str:
    text = re.sub("((ค่ะ)|((นะ)?ครับ)|(นะคะ)|เอ่อ)+", "", text)
    # text = thaispellcheck.check(text, autocorrect=True)
    return text

def pred_postprocess(listtext: List[str]) -> List[str]:

    if " " in listtext:
        listtext.remove(" ")
        
    clean_text = remove_longest_repeating_words(listtext)
    return clean_text

def tokenize_text(text: str, pred = False) -> list:
    if not isinstance(text, str):
        return []
    try:
        lang = detect(text)
    except:
        lang = 'en'
    if lang != 'en':
        text = th_preprocess(text)
        splited_text = deepcut.tokenize(text)
    else:
        eng_normalizer = EnglishTextNormalizer()
        text = eng_normalizer(text)
        text = en_preprocess(text)
        splited_text = text.split(" ")
        
    return pred_postprocess(splited_text) if pred else splited_text

In [7]:
from functools import partial
prediction_words = list(map(partial(tokenize_text, pred=True), prediction))
references_words = list(map(tokenize_text, references))

2023-11-24 15:08:42.937250: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


2/2 [==============================] - 0s 10ms/step
{'นี่': 1, 'ก็': 1, 'เป็น': 1, 'วัน': 1, 'Auxility': 1, ' ': 2, 'Day': 1, 'ที่': 1, 'ผม': 1, 'จะ': 1, 'รายงาน': 1}
2/2 [==============================] - 0s 10ms/step
{'แล้ว': 1, 'ก็': 1, 'อัปเดต': 1, 'เรื่อง': 1, 'ต่างๆ': 1, ' ': 2, 'ของ': 1, 'บริษัท Interfarm': 1, 'มาก': 1}
2/2 [==============================] - 0s 14ms/step
{'ทั้ง': 1, 'ของ': 2, 'คน': 1, 'แล้ว': 1, 'ก็': 1, 'สัตว์': 1, 'จาก': 1, 'ต้น': 1, 'น้ำ': 2, 'เดิน': 1, 'ตาย': 1}
1/1 [==============================] - 0s 23ms/step
{'ก็': 2, 'ขอ': 1, 'มา': 1, 'ปริศนา': 1, 'สุขภาพ': 1, 'กับ': 1, 'พวก': 1, 'องค์กร': 1}
1/1 [==============================] - 0s 30ms/step
{'จะ': 1, 'พูด': 1, 'ให้': 1, 'ฟัง': 1, 'ว่า': 2, 'เป็น': 1, 'อย่าง': 1, 'ไร': 1}
2/2 [==============================] - 0s 14ms/step
{'แล้ว': 1, 'ก็': 1, 'จาก': 1, 'ส้น': 1, 'น้ำ': 2, 'เป็น': 2, 'ปลาย': 1, 'อย่าง': 1, 'ไร': 1}
2/2 [==============================] - 0s 5ms/step
{'สำหรับ': 1, 'ครบ': 1, 'วงจร': 1, 

In [8]:
from functools import reduce

prediction_words_reduce = list(reduce(lambda a, b: a + b, prediction_words))
prediction_words_reduce = [word for word in prediction_words_reduce if not word in ["", " "]]
# references_words_reduce = list(reduce(lambda a, b: a + b, references_words))
references_words_reduce = [word for word in references_words[0] if not word in ["", " "]]

In [9]:
len(prediction_words_reduce)

7472

In [10]:
len(references_words_reduce)

6719

In [11]:
result, step, summary = isd(prediction_words_reduce, references_words_reduce, debug=True) 
print(f"WER: {result / len(references_words_reduce)}")

REF: สวัสดี สวัสดี นัก ลง ทุน ทุก ท่าน แล้ว ก็  คุณ ผู้ ชม ทุก ท่าน นี่     ก็ เป็น วัน Auxility    Day ที่ ผม จะ รายงาน           การ ล่าสุด ซึ่ง ก็ ล่าสุด ใน สินทรัพย์ รอบ    2 แล้ว ก็ อัปเดต เรื่อง ต่างๆ ของ บริษัท Interfarm มาก เอ๊ะ ไม่ เหนื่อย เลย  อย่า ท้อที่ ก็                      วิชัยทัศน์ ของ บริษัท ก็ เรา ต้องการ เป็น ผู้ นำ ผลิตภัณฑ์ สุขภาพ แบบ ครบ วงจร นะ ทั้ง ของ คน แล้ว ก็ ของ สัตว์ จาก ต้น น้ำ เดิน   ตาย  น้ำ ก็ ขอ มา  ปริศนา    สุขภาพ กับ พวก องค์กร ก็ จะ พูด ให้ ฟัง ว่า ว่า เป็น อย่าง ไร แล้ว ก็ จาก ส้น น้ำ เป็น   ปลาย น้ำ เป็น อย่าง ไร ไว้ สมัคร สำหรับ ครบ วงจร เครดิต ทาง       สุขภาพ เนี่ย ใน ใน วิสัย ทาง        ของ บริษัท ก็ คือ ว่า จะ  หมาย ความ ว่า ตั้งแต่ ที่ หัวพลิมิต  เลย ก็ คือ เรื่อง     เกี่ยว กับ นวัตกรรม ตอน นี้ งาน ของ ผม ที่ ทำ หลัก ๆ ก็ คือ เป็น การ ไป หานวัฏกรรม          ใหม่ ๆ ใน การ รักษา แล้ว ก็ การ ป้องกัน โรค แล้ว ก็ ถัด มา อัน นี้   ก็ จะ เป็น เรื่อง ของ ยา ซึ่ง ปัจจุบัน เรา มี โรง งาน ยา 2 โรง ที่ ผลิต ยา ที่ เรียก ว่า เจินาลิก หรือ Phenocytic

# Error Analysis

In [12]:
summary

,wer,insertions,deletions,substitutions,inserted_words,deleted_words,substituted_words
0,0.427891,320,1073,1554,"[กลาง, มัน, แต่, ของ, โฆษณา, ยิง, ราคา, อะไร, ...","[Yeah, ได้, เปิด, จะ, ที่, มี, อย่าง, จะ, ไป, ...","[(ปี, ตี), (กษัตริย์, บริษัท), (เป็น, ใช่), (น..."


In [13]:
summary.to_excel("summary.xlsx",
             sheet_name='summary')
inserted_words = pd.DataFrame({"inserted_words":summary.inserted_words.tolist()[0]})
deleted_words = pd.DataFrame({"deleted_words":summary.deleted_words.tolist()[0]})
substituted_words = pd.DataFrame({"substituted_words":summary.substituted_words.tolist()[0],
              "from": [i[0] for i in summary.substituted_words.tolist()[0]],
              "to": [i[1] for i in summary.substituted_words.tolist()[0]]})

# create a excel writer object
with pd.ExcelWriter("summary.xlsx") as writer:
    summary.to_excel(writer, sheet_name="summary", index=False)
    inserted_words.to_excel(writer, sheet_name="inserted_words", index=False)
    deleted_words.to_excel(writer, sheet_name="deleted_words", index=False)
    substituted_words.to_excel(writer, sheet_name="substituted_words", index=False)

In [ ]:
import jiwer
jiwer.cer(hypothesis="".join(prediction_words_reduce), reference="".join(references_words_reduce))

0.374805279986882